In [34]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


In [ ]:
#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")



In [66]:
import pandas as pd
df = pd.read_pickle('../news_row/2021-9.pkl')

In [67]:
def text_wrap(text):
    if not text:
        text = ''
    return text.replace('######','').replace('<p>', '').replace('</p>', '').replace('&nbsp;', '') + '  '



In [68]:
rs = pd.DataFrame()
rs['title'] = df['title']
rs['text'] = df['summary'].map(text_wrap)+df['content'].map(text_wrap)

In [69]:
rs

,title,text
0,嘉元科技出现1笔大宗交易 折价率为0.24%,"8月31日讯，嘉元科技出现1笔大宗交易。成交价格为111.49元,成交金额为303.7099..."
1,中简科技出现1笔大宗交易 折价率为19.16%,"8月31日讯，中简科技出现1笔大宗交易。成交价格为38.48元,成交金额为1299.85万元..."
2,美国培训空姐练武术对付愤怒乘客,民航资源网2021年8月31日消息：据央视网报道，近日，美国航司发现，航班上因“愤怒的乘客”...
3,重庆机场集团换帅，总经理戴科接棒,民航资源网2021年8月31日消息：8月30日，重庆机场集团发布关于公司董事长发生变动的公告...
4,机票“秒退”、行李收费差异化……9月起乘机出行将有这些变化,机票退款慢、投诉渠道不通畅、行李托运费用贵……针对旅客乘机出行的难点、痛点，民航局颁布《公共...
...,...,...
147560,​李克强会见2020和2021年度中国政府友谊奖获奖外国专家,李克强会见2020和2021年度中国政府友谊奖获奖外国专家韩正参加会见国务院总理李克强9月3...
147561,李克强会见28国新任驻华使节,李克强会见28国新任驻华使节强调中国的发展始终在改革中推进、在开放中前行在中华人民共和国成立...
147562,9月30日电，美国财长耶伦称，在10月18日触及债务上限的情况下，美国(距离发生债务违约)可...,9月30日电，美国财长耶伦称，在10月18日触及债务上限的情况下，美国(距离发生债务违约)可...
147563,美股三大股指悉数转跌,9月30日电，美股高开低走，三大股指悉数转跌，道指现跌0.88%，标普500指数跌0.5%；...


In [70]:
import gensim
import jieba
import pandas as pd
import os
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [74]:
def cut(t):
    # stopwords = [line[:-1] for line in open('stopwords.txt', encoding='utf-8')]
    result = []
    for i in t:
        i_c = jieba.cut(i)
        i_c = list(i_c)
        # i_c = [i for i in i_c if i not in stopwords]
        result.append(i_c)
    return result

def prepare_doc2vec(data):
    result = []
    for i, t in enumerate(data):
        l = len(t)
        if l == 0:
            continue
        t[l-1] = t[l-1].strip()
        documents = TaggedDocument(t, [i])
        result.append(documents)
    return result

def train(data, size=300):
    model = Doc2Vec(data, window=5, vector_size=size, min_count=1, workers=12)
    model.train(data, total_examples=model.corpus_count, epochs=10)
    return model



In [75]:
m = train(prepare_doc2vec(cut(rs['title'])))
m.dv.vectors

array([[-0.01718042,  0.0315343 ,  0.00023681, ..., -0.03623222,
         0.0331271 , -0.02939493],
       [-0.02119157,  0.01837072,  0.02011241, ..., -0.02334457,
        -0.01449429, -0.01363768],
       [-0.04252871, -0.00404154, -0.02762956, ..., -0.04628174,
        -0.04922506, -0.0807714 ],
       ...,
       [ 0.00780422, -0.00705105,  0.08866067, ..., -0.21525773,
         0.07330853,  0.17235218],
       [-0.0094067 ,  0.02352455, -0.00801573, ..., -0.02858238,
         0.01358339, -0.02961554],
       [-0.02622276,  0.00687354,  0.00039805, ..., -0.0353792 ,
         0.00774546, -0.02699653]], dtype=float32)

In [76]:
import numpy as np
np.save('2021-9.npy', m.dv.vectors)